In [1]:
import pandas as pd

In [55]:
org_df = pd.read_csv("/data/lux70/data/pfam/organism_counts.csv")
org_df.head()

,organism_index,organism_id,counts
0,0,SYNS9,2126
1,1,SCHAZ,4805
2,2,9ENTE,108317
3,3,APHAT,14121
4,4,9GAMM,2601336


In [33]:
!head /data/lux70/data/uniref90/speclist_parse.txt 

AADNV V  648330: N=Aedes albopictus densovirus (isolate Boublik/1994)
AAV2  V   10804: N=Adeno-associated virus 2
AAV2S V  648242: N=Adeno-associated virus 2 (isolate Srivastava/1982)
ABABO E 3053407: N=Abaeis boisduvaliana
ABAMA E  118452: N=Abacion magnum
ABAME E 3053408: N=Abaeis mexicana
ABANI E   72259: N=Abaeis nicippe
ABAPA E  102642: N=Abax parallelepipedus
ABAST E  392897: N=Abalistes stellaris
ABBRI E   75332: N=Abbottina rivularis


In [34]:
df = pd.read_csv("/data/lux70/data/uniref90/speclist_parse.txt", sep=": N=",header=None)
df.columns = ['uniref_code','name']

In [35]:
df['uniref_identifier'] = df.uniref_code.str[:5]
df['uniref_code'] = df.uniref_code.str[7:].astype(int)

In [36]:
df.head()

,uniref_code,name,uniref_identifier
0,648330,Aedes albopictus densovirus (isolate Boublik/1...,AADNV
1,10804,Adeno-associated virus 2,AAV2
2,648242,Adeno-associated virus 2 (isolate Srivastava/1...,AAV2S
3,3053407,Abaeis boisduvaliana,ABABO
4,118452,Abacion magnum,ABAMA


In [7]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()
#ncbi.update_taxonomy_database()  # Download NCBI taxonomy

In [30]:
for taxon_id in df['uniref_code'][1:40:4]:
    print(f"{taxon_id} ======")
    lineage = ncbi.get_lineage(taxon_id)
    names = ncbi.get_taxid_translator(lineage)
    
    # Print higher-order taxonomy names
    print(len(lineage), [names[taxid] for taxid in lineage])

10804 ======
12 ['root', 'Viruses', 'Monodnaviria', 'Shotokuvirae', 'Cossaviricota', 'Quintoviricetes', 'Piccovirales', 'Parvoviridae', 'Parvovirinae', 'Dependoparvovirus', 'Dependoparvovirus primate1', 'adeno-associated virus 2']
3053408 ======
31 ['root', 'cellular organisms', 'Eukaryota', 'Opisthokonta', 'Metazoa', 'Eumetazoa', 'Bilateria', 'Protostomia', 'Ecdysozoa', 'Panarthropoda', 'Arthropoda', 'Mandibulata', 'Pancrustacea', 'Hexapoda', 'Insecta', 'Dicondylia', 'Pterygota', 'Neoptera', 'Endopterygota', 'Amphiesmenoptera', 'Lepidoptera', 'Glossata', 'Neolepidoptera', 'Heteroneura', 'Ditrysia', 'Obtectomera', 'Papilionoidea', 'Pieridae', 'Coliadinae', 'Abaeis', 'Abaeis mexicana']
75332 ======
30 ['root', 'cellular organisms', 'Eukaryota', 'Opisthokonta', 'Metazoa', 'Eumetazoa', 'Bilateria', 'Deuterostomia', 'Chordata', 'Craniata', 'Vertebrata', 'Gnathostomata', 'Teleostomi', 'Euteleostomi', 'Actinopterygii', 'Actinopteri', 'Neopterygii', 'Teleostei', 'Osteoglossocephalai', 'Clupeo

In [51]:
ranks

{1: 'no rank',
 2759: 'superkingdom',
 3193: 'clade',
 3313: 'subclass',
 3318: 'family',
 3319: 'genus',
 33090: 'kingdom',
 35493: 'phylum',
 58019: 'class',
 58023: 'clade',
 58024: 'clade',
 78536: 'clade',
 97170: 'species',
 131221: 'subphylum',
 131567: 'no rank',
 1437180: 'clade',
 1446380: 'order',
 2821352: 'clade'}

In [45]:
def parse_level(taxon_id, level):
    try:
        lineage = ncbi.get_lineage(taxon_id)
        names = ncbi.get_taxid_translator(lineage)
         
        ranks = ncbi.get_rank(lineage)
        
        # Find the kingdom in the lineage
        levels = [names[taxid] for taxid in lineage if ranks[taxid] == level]
        return levels[0]
        
    except:
        return ""

In [41]:
import warnings
warnings.filterwarnings("ignore")


In [44]:
import numpy as np

np.unique(df.kingdom.values, return_counts=True)

(array(['', 'Bamfordvirae', 'Fungi', 'Heunggongvirae', 'Loebvirae',
        'Metazoa', 'Orthornavirae', 'Pararnavirae', 'Sangervirae',
        'Shotokuvirae', 'Trapavirae', 'Viridiplantae', 'Zilligvirae'],
       dtype=object),
 array([ 5540,   176,  1102,   380,    14, 13270,  1744,   363,    10,
          234,     2,  4802,    14]))

In [48]:
import warnings
warnings.filterwarnings("ignore")
df['superkingdom'] = df.uniref_code.map(lambda x: parse_level(x, "superkingdom"))

In [49]:
np.unique(df.superkingdom, return_counts=True)

(array(['', 'Archaea', 'Bacteria', 'Eukaryota', 'Viruses'], dtype=object),
 array([  128,   330,  4259, 19855,  3079]))

In [53]:
import warnings
warnings.filterwarnings("ignore")
# df['kingdom'] = df.uniref_code.map(lambda x: parse_kingdom(x, "kingdom"))
df['phylum'] = df.uniref_code.map(lambda x: parse_level(x, "phylum"))
df['class'] = df.uniref_code.map(lambda x: parse_level(x, "class"))
df['order'] = df.uniref_code.map(lambda x: parse_level(x, "order"))
df['family'] = df.uniref_code.map(lambda x: parse_level(x, "family"))
df['genus'] = df.uniref_code.map(lambda x: parse_level(x, "genus"))
df['species'] = df.uniref_code.map(lambda x: parse_level(x, "species"))

In [58]:
df = df.set_index("uniref_identifier").join(org_df.set_index("organism_id"))

In [60]:
df.head()

,uniref_code,name,kingdom,superkingdom,phylum,class,order,family,genus,species,organism_index,counts
uniref_identifier,,,,,,,,,,,,
AADNV,648330,Aedes albopictus densovirus (isolate Boublik/1...,Shotokuvirae,Viruses,Cossaviricota,Quintoviricetes,Piccovirales,Parvoviridae,Brevihamaparvovirus,Brevihamaparvovirus dipteran1,3491.0,2.0
AAV2,10804,Adeno-associated virus 2,Shotokuvirae,Viruses,Cossaviricota,Quintoviricetes,Piccovirales,Parvoviridae,Dependoparvovirus,Dependoparvovirus primate1,NaN,NaN
AAV2S,648242,Adeno-associated virus 2 (isolate Srivastava/1...,Shotokuvirae,Viruses,Cossaviricota,Quintoviricetes,Piccovirales,Parvoviridae,Dependoparvovirus,Dependoparvovirus primate1,3489.0,4.0
ABABO,3053407,Abaeis boisduvaliana,Metazoa,Eukaryota,Arthropoda,Insecta,Lepidoptera,Pieridae,Abaeis,Abaeis boisduvaliana,NaN,NaN
ABAMA,118452,Abacion magnum,Metazoa,Eukaryota,Arthropoda,Diplopoda,Callipodida,Callipodidae,Abacion,Abacion magnum,NaN,NaN


In [62]:
# df.to_csv("/data/lux70/data/uniref90/all_specs_parsed.csv", index=True)

In [64]:
df = df.reset_index()

In [68]:
included_organisms_df = df[~df.organism_index.isna()]

In [69]:
included_organisms_df['organism_index'] = included_organisms_df.organism_index.astype(int)

In [70]:
included_organisms_df['counts'] = included_organisms_df.counts.astype(int)

In [71]:
included_organisms_df.head()

,uniref_identifier,uniref_code,name,kingdom,superkingdom,phylum,class,order,family,genus,species,organism_index,counts
0,AADNV,648330,Aedes albopictus densovirus (isolate Boublik/1...,Shotokuvirae,Viruses,Cossaviricota,Quintoviricetes,Piccovirales,Parvoviridae,Brevihamaparvovirus,Brevihamaparvovirus dipteran1,3491,2
2,AAV2S,648242,Adeno-associated virus 2 (isolate Srivastava/1...,Shotokuvirae,Viruses,Cossaviricota,Quintoviricetes,Piccovirales,Parvoviridae,Dependoparvovirus,Dependoparvovirus primate1,3489,4
22,ABIDE,46125,Abiotrophia defectiva,,Bacteria,Bacillota,Bacilli,Lactobacillales,Aerococcaceae,Abiotrophia,Abiotrophia defectiva,2236,2411
40,ABLVB,446561,Australian bat lyssavirus (isolate Bat/AUS/1996),Orthornavirae,Viruses,Negarnaviricota,Monjiviricetes,Mononegavirales,Rhabdoviridae,Lyssavirus,Lyssavirus australis,3422,8
43,ABPVR,1217067,Acute bee paralysis virus (strain Rothamsted),Orthornavirae,Viruses,Pisuviricota,Pisoniviricetes,Picornavirales,Dicistroviridae,Aparavirus,Aparavirus apisacutum,3093,6


In [72]:
# included_organisms_df.to_csv("/data/lux70/data/pfam/organism_hierarchy.csv", index=False)

In [74]:
np.unique(included_organisms_df.kingdom, return_counts=True)

(array(['', 'Bamfordvirae', 'Fungi', 'Heunggongvirae', 'Loebvirae',
        'Metazoa', 'Orthornavirae', 'Pararnavirae', 'Sangervirae',
        'Shotokuvirae', 'Trapavirae', 'Viridiplantae', 'Zilligvirae'],
       dtype=object),
 array([1548,   61,  371,  145,    8,  642,  267,   62,    5,   95,    1,
         166,    7]))

In [76]:
np.unique(included_organisms_df.superkingdom, return_counts=True)

(array(['', 'Archaea', 'Bacteria', 'Eukaryota', 'Viruses'], dtype=object),
 array([  33,  109, 1247, 1295,  694]))

In [77]:
np.unique(included_organisms_df.phylum, return_counts=True)

(array(['', 'Acidobacteriota', 'Actinomycetota', 'Annelida', 'Apicomplexa',
        'Aquificota', 'Armatimonadota', 'Arthropoda', 'Artverviricota',
        'Ascomycota', 'Bacillariophyta', 'Bacillota', 'Bacteroidota',
        'Basidiomycota', 'Bdellovibrionota', 'Blastocladiomycota',
        'Brachiopoda', 'Bryozoa', 'Caldisericota', 'Campylobacterota',
        'Candidatus Cloacimonadota', 'Candidatus Micrarchaeota',
        'Candidatus Parvarchaeota', 'Candidatus Thermoplasmatota',
        'Chlamydiota', 'Chlorobiota', 'Chloroflexota', 'Chlorophyta',
        'Chordata', 'Chytridiomycota', 'Ciliophora', 'Cnidaria',
        'Coprothermobacterota', 'Cossaviricota', 'Cressdnaviricota',
        'Cryptomycota', 'Cyanobacteriota', 'Deferribacterota',
        'Deinococcota', 'Dictyoglomota', 'Discosea', 'Duplornaviricota',
        'Echinodermata', 'Elusimicrobiota', 'Endomyxa', 'Euglenozoa',
        'Euryarchaeota', 'Evosea', 'Fibrobacterota', 'Foraminifera',
        'Fornicata', 'Fusobacteri